In [1]:
import numpy as np
import plotly.express as px
import tensorflow as tf
from utils.preprocessing import *
import sys

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

from keras.backend import *
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior() # enables np methods on tensors

In [2]:
#testing_dir = '../data/C-NMC_Leukemia/testing_data/C-NMC_test_final_phase_data' ### WE DO NOT USE THIS
training_dir_base = f'../data/C-NMC_Leukemia/training_data/' 
fold_0_all_path = 'fold_0/all'
fold_0_hem_path = 'fold_0/hem'
fold_1_all_path = 'fold_1/all'
fold_1_hem_path = 'fold_1/hem'

In [3]:
### The code for this lives in notebooks.utils.preprocessing
### Get grayscale images as tensors, and labels for both classes
img_train_cancer = get_gray_images(training_dir_base+fold_0_all_path)
img_train_healthy = get_gray_images(training_dir_base+fold_0_hem_path)
train_health_labels = [0 for _ in range(len(img_train_healthy))]
train_cancer_labels = [1 for _ in range(len(img_train_cancer))]

## Random Forests

In [8]:
def randomForestAccuracy(img_cancer, img_healthy, labels_healthy, labels_cancer, nest=100, mssplit=2, mleaf=1):
    ### For random forests, X is our combined image set and y is our combined label set.
    X =  np.array(img_cancer + img_healthy)
    y = np.array(labels_cancer + labels_healthy)

    ### train-test-split our data, and initialize the forest.
    X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state=100)

    forest = RandomForestClassifier(n_estimators=nest, random_state=100, min_samples_split=mssplit, min_samples_leaf=mleaf)

    ### The random forests algorithm expects 2 dimensional data at the most.
    ### We can accomplish this on our 4-dimensional data by multiplying the last 3 dimensions together to flatten the data
    X_shape = X_train.shape
    X_tshape = X_test.shape
    X_train = X_train.reshape(X_shape[0], X_shape[1] * X_shape[2] * X_shape[3])
    X_test = X_test.reshape(X_tshape[0], X_tshape[1] * X_tshape[2] * X_tshape[3])

    ### Fit the random forest, try it on the test data, and assess the model's accuracy
    forest.fit(X_train,y_train)
    predictions = forest.predict(X_test)
    return metrics.accuracy_score(y_test, predictions), forest, predictions, y_test

In [9]:
accuracy_original, rf_model, predictions, labels = randomForestAccuracy(img_train_cancer, img_train_healthy, train_health_labels, train_cancer_labels)

In [13]:
confusion_matrix = metrics.confusion_matrix(labels, predictions)
px.imshow(
    confusion_matrix, text_auto=True,
    labels=dict(x="Confusion Matrix Random Forest"),
    x=['Positive', 'Negative'],
    y=['Positive', 'Negative']
)

In [8]:
### Random forests, color
### We will repeat the steps above, with color images instead.
img_train_cancer = get_gray_images(training_dir_base+fold_0_all_path, 128)
img_train_healthy = get_gray_images(training_dir_base+fold_0_hem_path, 128)

In [13]:
accuracy_128, rf_model_128 = randomForestAccuracy(img_train_cancer, img_train_healthy, train_health_labels, train_cancer_labels)

In [14]:
### I find that there is a negligible (0.4%) difference between full size and 128x128 downscaled images.
### Because downscaled images process faster, I will use those
print(accuracy_original)
print(accuracy_128)

0.8153078202995009
0.8153078202995009


In [16]:
### Assemble combinations of hyperparameters for tuning.
from itertools import product
_n_estimators = [50, 100, 150]
_min_samples_split = [2, 5, 10]
_min_samples_leaf = [1, 2, 5]
list_of_lists = [_n_estimators, _min_samples_split, _min_samples_leaf]
all_combinations = list(product(*list_of_lists))

In [17]:
### Find the highest accuracy combination of hyperparameters.
max_accuracy = -1
max_accuracy_params = []
for combination in all_combinations:
    accuracy = randomForestAccuracy(img_train_cancer, img_train_healthy, train_health_labels, train_cancer_labels, 
                                    nest=combination[0], mssplit=combination[1], mleaf=combination[2])
    
    if accuracy > max_accuracy:
        max_accuracy = accuracy
        max_accuracy_params = combination

In [18]:
print(max_accuracy)
print(max_accuracy_params)

0.8202995008319468
(150, 2, 1)


# Perceptron

Perceptron: Grayscale Images

In [14]:
grayCancer = get_gray_images(training_dir_base+fold_0_all_path)
grayHealthy = get_gray_images(training_dir_base+fold_0_hem_path)

trainCancerLabels = [1 for _ in range(len(grayCancer))]
trainHealthLabels = [0 for _ in range(len(grayHealthy))]

images, labels = grayCancer + grayHealthy, trainCancerLabels + trainHealthLabels # concatenate cancer and healthy images, as well as their labels into combined image and label vectors
images, labels = np.array(images), np.array(labels)

# convert to np arrays for training
grayImages = np.array(images).reshape(len(images), 450, 450)
grayLabels = np.array(labels)

In [15]:
# shuffle labels and images in unison so they maintain their image-label correspondance
randomize = np.arange(len(grayImages))
np.random.shuffle(randomize)
grayImages = grayImages[randomize]
grayLabels = grayLabels[randomize]

In [16]:
with tf.device('/CPU:0'): # tf uses GPU by default, use this if your config is out of whack like mine is -C.J.
                            # otherwise use - with tf.device('/GPU:0) or remove code from under with statement
    model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape = [450, 450]),
        tf.keras.layers.Dense(1, activation='sigmoid'), # Dense operation is output = activation(dot(input, kernel) + bias)
    ])

    model.compile(optimizer='adam', 
                loss=tf.keras.losses.BinaryCrossentropy(),                
                metrics=['accuracy'])

    #print(images.shape, labels.shape)

    model.fit(grayImages, grayLabels, epochs = 20, batch_size = 10)

Epoch 1/20


2022-12-04 18:39:29.889784: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


201/201 [==============================] - 1s 2ms/step - loss: 100.9833 - accuracy: 0.6828
Epoch 2/20
201/201 [==============================] - 0s 1ms/step - loss: 114.5027 - accuracy: 0.6983
Epoch 3/20
201/201 [==============================] - 0s 1ms/step - loss: 67.6641 - accuracy: 0.7507
Epoch 4/20
201/201 [==============================] - 0s 1ms/step - loss: 90.7540 - accuracy: 0.7443
Epoch 5/20
201/201 [==============================] - 0s 1ms/step - loss: 63.1239 - accuracy: 0.7622
Epoch 6/20
201/201 [==============================] - 0s 1ms/step - loss: 72.7821 - accuracy: 0.7652
Epoch 7/20
201/201 [==============================] - 0s 1ms/step - loss: 102.5404 - accuracy: 0.7652
Epoch 8/20
201/201 [==============================] - 0s 1ms/step - loss: 53.2697 - accuracy: 0.7902
Epoch 9/20
201/201 [==============================] - 0s 1ms/step - loss: 106.3919 - accuracy: 0.7707
Epoch 10/20
201/201 [==============================] - 0s 2ms/step - loss: 41.6441 - accuracy: 0.8

In [17]:
### Create testing data
grayCancerTest = get_gray_images(training_dir_base+fold_1_all_path)
grayHealthyTest = get_gray_images(training_dir_base+fold_1_hem_path)

testCancerLabels = [1 for _ in range(len(grayCancerTest))]
testHealthLabels = [0 for _ in range(len(grayHealthyTest))]

images, labels = grayCancerTest + grayHealthyTest, testCancerLabels + testHealthLabels # concatenate cancer and healthy images, as well as their labels into combined image and label vectors
images, labels = np.array(images), np.array(labels)

# convert to np arrays for training
grayImagesTest = np.array(images).reshape(len(images), 450, 450)
grayLabelsTest = np.array(labels)

In [18]:
### Test model on test data
model.evaluate(grayImagesTest, grayLabelsTest)

63/63 [==============================] - 0s 6ms/step - loss: 138.1670 - accuracy: 0.7113


[138.16700744628906, 0.7112886905670166]

Perceptron: HSV Images

In [19]:
hsvCancer = get_hsv_images(training_dir_base+fold_0_all_path)
hsvHealthy = get_hsv_images(training_dir_base+fold_0_hem_path)

trainCancerLabels = [1 for _ in range(len(hsvCancer))]
trainHealthLabels = [0 for _ in range(len(hsvHealthy))]

images, labels = hsvCancer + hsvHealthy, trainCancerLabels + trainHealthLabels # concatenate cancer and healthy images, as well as their labels into combined image and label vectors
images, labels = np.array(images), np.array(labels)

# convert to np arrays for training
hsvImages = np.array(images).reshape(len(images), 450, 450, 3)
hsvLabels = np.array(labels)
#plt.imshow(hsvImages[0])

In [20]:
# shuffle labels and images in unison so they maintain their image-label correspondance
randomize = np.arange(len(hsvImages))
np.random.shuffle(randomize)
hsvImages = hsvImages[randomize]
hsvLabels = hsvLabels[randomize]

In [21]:
with tf.device('/CPU:0'): 
    model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape = [450, 450, 3]),
        tf.keras.layers.Dense(1, activation='sigmoid'), # Dense operation is output = activation(dot(input, kernel) + bias)
    ])

    model.compile(optimizer='adam', 
                loss=tf.keras.losses.BinaryCrossentropy(),                
                metrics=['accuracy'])

    model.fit(hsvImages, hsvLabels, epochs = 10, batch_size = 10)

Epoch 1/10
201/201 [==============================] - 1s 5ms/step - loss: 198.8432 - accuracy: 0.6903
Epoch 2/10
201/201 [==============================] - 1s 5ms/step - loss: 131.0860 - accuracy: 0.7138
Epoch 3/10
201/201 [==============================] - 1s 5ms/step - loss: 178.6445 - accuracy: 0.7228
Epoch 4/10
201/201 [==============================] - 1s 5ms/step - loss: 105.0891 - accuracy: 0.7438
Epoch 5/10
201/201 [==============================] - 1s 6ms/step - loss: 101.1356 - accuracy: 0.7522
Epoch 6/10
201/201 [==============================] - 1s 5ms/step - loss: 81.9695 - accuracy: 0.7817
Epoch 7/10
201/201 [==============================] - 1s 5ms/step - loss: 109.5454 - accuracy: 0.7522
Epoch 8/10
201/201 [==============================] - 1s 5ms/step - loss: 116.4337 - accuracy: 0.7757
Epoch 9/10
201/201 [==============================] - 1s 5ms/step - loss: 76.3094 - accuracy: 0.8042
Epoch 10/10
201/201 [==============================] - 1s 5ms/step - loss: 96.8500 -

In [22]:
### Create testing data
hsvCancerTest = get_hsv_images(training_dir_base+fold_1_all_path)
hsvHealthyTest = get_hsv_images(training_dir_base+fold_1_hem_path)

testCancerLabels = [1 for _ in range(len(hsvCancerTest))]
testHealthLabels = [0 for _ in range(len(hsvHealthyTest))]

images, labels = hsvCancerTest + hsvHealthyTest, testCancerLabels + testHealthLabels # concatenate cancer and healthy images, as well as their labels into combined image and label vectors
images, labels = np.array(images), np.array(labels)

# convert to np arrays for training
hsvImagesTest = np.array(images).reshape(len(images), 450, 450, 3)
hsvLabelsTest = np.array(labels)

In [23]:
### Test model on test data
model.evaluate(hsvImagesTest, hsvLabelsTest)

63/63 [==============================] - 1s 20ms/step - loss: 134.4878 - accuracy: 0.7048


[134.48776245117188, 0.7047951817512512]

Perceptron: Saturated Images

In [19]:
satCancer = get_saturated_images(training_dir_base+fold_0_all_path)
satHealthy = get_saturated_images(training_dir_base+fold_0_hem_path)

trainCancerLabels = [1 for _ in range(len(satCancer))]
trainHealthLabels = [0 for _ in range(len(satHealthy))]

images, labels = satCancer + satHealthy, trainCancerLabels + trainHealthLabels # concatenate cancer and healthy images, as well as their labels into combined image and label vectors
images, labels = np.array(images), np.array(labels)

# convert to np arrays for training
satImages = np.array(images).reshape(len(images), 450, 450, 3)
satLabels = np.array(labels)

In [20]:
# shuffle labels and images in unison so they maintain their image-label correspondance
randomize = np.arange(len(satImages))
np.random.shuffle(randomize)
satImages = satImages[randomize]
satLabels = satLabels[randomize]

In [21]:
with tf.device('/CPU:0'): 
    model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape = [450, 450, 3]),
        tf.keras.layers.Dense(1, activation='sigmoid'), # Dense operation is output = activation(dot(input, kernel) + bias)
    ])

    model.compile(optimizer='adam', 
                loss=tf.keras.losses.BinaryCrossentropy(),                
                metrics=['accuracy'])

    model.fit(satImages, satLabels, epochs = 10, batch_size = 10)

Epoch 1/10
201/201 [==============================] - 1s 5ms/step - loss: 474.2840 - accuracy: 0.6813
Epoch 2/10
201/201 [==============================] - 1s 5ms/step - loss: 321.0898 - accuracy: 0.7088
Epoch 3/10
201/201 [==============================] - 1s 6ms/step - loss: 305.6201 - accuracy: 0.7597
Epoch 4/10
201/201 [==============================] - 1s 6ms/step - loss: 286.0834 - accuracy: 0.7502
Epoch 5/10
201/201 [==============================] - 1s 6ms/step - loss: 324.7443 - accuracy: 0.7687
Epoch 6/10
201/201 [==============================] - 1s 6ms/step - loss: 435.4881 - accuracy: 0.7797
Epoch 7/10
201/201 [==============================] - 1s 5ms/step - loss: 379.8261 - accuracy: 0.7677
Epoch 8/10
201/201 [==============================] - 1s 5ms/step - loss: 224.7224 - accuracy: 0.7972
Epoch 9/10
201/201 [==============================] - 1s 5ms/step - loss: 92.1616 - accuracy: 0.8506
Epoch 10/10
201/201 [==============================] - 1s 5ms/step - loss: 132.1814

Perceptron: Saturated Images

In [22]:
### Create testing data
satCancerTest = get_saturated_images(training_dir_base+fold_1_all_path)
satHealthyTest = get_saturated_images(training_dir_base+fold_1_hem_path)

testCancerLabels = [1 for _ in range(len(satCancerTest))]
testHealthLabels = [0 for _ in range(len(satHealthyTest))]

images, labels = satCancerTest + satHealthyTest, testCancerLabels + testHealthLabels # concatenate cancer and healthy images, as well as their labels into combined image and label vectors
images, labels = np.array(images), np.array(labels)

# convert to np arrays for training
satImagesTest = np.array(images).reshape(len(images), 450, 450, 3)
satLabelsTest = np.array(labels)
### usually works, currently expirencing resource exhausted error- plz try on your machine if you pull this -C.J.

In [24]:
### Test model on test data
model.evaluate(satImagesTest, satLabelsTest)

63/63 [==============================] - 1s 19ms/step - loss: 327.3982 - accuracy: 0.7488


[327.3981628417969, 0.7487512230873108]

In [44]:
predictions = model.predict(satImagesTest)

63/63 [==============================] - 2s 28ms/step


In [45]:
### Attempt at rounding the final values so that all values are either a 1 or 0

for i in range(len(predictions)):
    if predictions[i] < 1 and predictions[i] > 0:
        if predictions[i] > .5:
            predictions[i] = 1.0
        else:
            predictions[i] = 0.0

In [46]:
for i in range(len(predictions)):
    if predictions[i] < 1 and predictions[i] > 0:
        print(predictions[i])

In [47]:
metrics.accuracy_score(satLabelsTest, predictions)

0.7487512487512488

In [48]:
confusion_matrix = metrics.confusion_matrix(satLabelsTest, predictions)
px.imshow(
    confusion_matrix, text_auto=True,
    labels=dict(x="Confusion Matrix Neural Net"),
    x=['Positive', 'Negative'],
    y=['Positive', 'Negative']
)